# githru-llm-engine

## 개요

### 목표 : githru-ext 에 LLM 봇 추가

### 데모1 : 커밋 히스토리 요약

- 예시 질문) 최근 일주일간 커밋 작업내용 요약해줘.

- 예시 응답) dt1~dt2 기간동안 커미터A 가 X 내용을 Y 로 수정했습니다.

### 데모2 : 커밋 diff 기반 커밋 타이틀 추천

- 참고자료
  - https://plugins.jetbrains.com/plugin/21056-codegpt
  - https://www.npmjs.com/package/opencommit
  - https://marketplace.visualstudio.com/items?itemName=Continue.continue
    - https://github.com/continuedev/continue/blob/main/core/context/providers/DocsContextProvider.ts#L100

- input) git diff HEAD HEAD~1

- 예시 질문) git diff + 커밋 제목 요청 프롬프트 + few shot 프롬프트

- 예시 응답) feat(브랜치명): commitNodeListInCluster 처리 방식 변경


### 데모3 : 커밋 diff 기반 코드리뷰

- 참고자료
  - https://itchipmunk.tistory.com/592
  - https://github.com/PickleBoxer/dev-chatgpt-prompts
  - https://www.promptingguide.ai/kr/techniques/cot

- 예시 질문) git diff + 코드리뷰 요청 프롬프트

- 예시 응답) 변경사항 요약 + 제안 사항




---
## 데모1 : 커밋 요약

### 1. 최근 git log 추출

```bash
git for-each-ref --sort=-committerdate refs/heads/ --format='%(committerdate:short) %(refname:short)'
git --no-pager log --since="2023-05-01" --before="2023-05-14"
최근 작업 브랜치 / 최근 작업일자 추출
```

### 2. 요약

```txt
아래 git log 내용을 요약해줘.
사용자별로 묶고, 그 아래에 날짜별로 묶어서 요약해줘.
동일날짜에서 작업한 내용들은 리스트로 구분해서 표현해줘.
사용자의 구분값은 이메일주소를 기준으로 해줘.
```

```txt
각 사용자별 커밋 수를 Bar 차트로 표현해줘
Bar 게이지와 함께 커밋 수도 같이 표현해줘
차트는 아스키 아트로 표현해줘
tab indent를 맞춰줘
차트를 만드는 소스코드는 답변에서 생략해줘
```



---
## 데모2 : 커밋 타이틀 추천

### 1. 커밋 diff 뽑기

```bash
git --no-pager diff 872ab1a37c8e5ce602ece69621df1fb61156eaf0 9d1bda31ef59a2ce7d1fa0e6d139beaeecf0d868
git --no-pager diff HEAD HEAD~1
git --no-pager diff --cached
```

### 2. 커밋 diff 기반 제목 요약 (+커밋 제목 규칙 few shot)

현재 사용하고 있는 커밋 제목 규칙은 아래와 같아.

```txt
git diff 작업내용 1 : <브랜치명> 브랜치에서 AAA버그를 수정함
commit title 1 (en) : fix(<브랜치명>): Bugfix AAA
commit title 1 (ko) : fix(<브랜치명>): AAA 버그를 수정

git diff 작업내용 2 : <브랜치명> 브랜치에서 BBB 기능을 추가함
commit title 2 (en) : feat(<브랜치명>): Implement BBB
commit title 2 (ko) : feat(<브랜치명>): BBB 기능 구현

git diff 작업내용 3 : <브랜치명> 브랜치에서 CCC 설정를 변경함. 비즈니스 로직 변경 없음.
commit title 3 (en) : chore(<브랜치명>): Change configuration CCC old -> new.
commit title 3 (ko) : chore(<브랜치명>): CCC 설정 변경
```

아래 git diff 를 보고 위 규칙대로 커밋 제목을 만들어줘.
한글로 만들어줘.

```bash
diff --git a/packages/view/src/components/Detail/Detail.hook.tsx b/packages/view/src/components/Detail/Detail.hook.tsx
index ac2a121..054a07e 100644
--- a/packages/view/src/components/Detail/Detail.hook.tsx
+++ b/packages/view/src/components/Detail/Detail.hook.tsx
@@ -13,9 +13,9 @@ const useToggleHook = (init = false): UseToggleHook => {

 export const useCommitListHide = (commitNodeListInCluster: CommitNode[]) => {
   const list = getSummaryCommitList(commitNodeListInCluster).reverse();
- const strech = commitNodeListInCluster.reverse();
+ const strech = commitNodeListInCluster.slice(5, commitNodeListInCluster.length).reverse();
   const [toggle, handleToggle] = useToggleHook();
- const commitNodeList = toggle ? strech : list;
+ const commitNodeList = toggle ? [...list, ...strech] : list;

   return {
     toggle,
```




## 데모3 : 코드리뷰

### 1. 커밋 diff 뽑기

`git --no-pager diff HEAD 4108be1a2f3234e3b3f474addc88458259119f7d`

### 2. 코드 리뷰

`아래 제공된 코드 변경점을 바탕으로 논리적 또는 보안 문제가 있는지를 검토하고 권장 사항 목록을 제공해줘.`

```bash
diff --git a/packages/view/src/components/Detail/Detail.hook.tsx b/packages/view/src/components/Detail/Detail.hook.tsx
index ac2a121..054a07e 100644
--- a/packages/view/src/components/Detail/Detail.hook.tsx
+++ b/packages/view/src/components/Detail/Detail.hook.tsx
@@ -13,9 +13,9 @@ const useToggleHook = (init = false): UseToggleHook => {

 export const useCommitListHide = (commitNodeListInCluster: CommitNode[]) => {
   const list = getSummaryCommitList(commitNodeListInCluster).reverse();
- const strech = commitNodeListInCluster.reverse();
+ const strech = commitNodeListInCluster.slice(5, commitNodeListInCluster.length).reverse();
   const [toggle, handleToggle] = useToggleHook();
- const commitNodeList = toggle ? strech : list;
+ const commitNodeList = toggle ? [...list, ...strech] : list;

   return {
     toggle,
diff --git a/packages/vscode/src/extension.ts b/packages/vscode/src/extension.ts
index 10e67a4..284870b 100644
--- a/packages/vscode/src/extension.ts
+++ b/packages/vscode/src/extension.ts
@@ -34,7 +34,7 @@ export async function activate(context: vscode.ExtensionContext) {

   const disposable = vscode.commands.registerCommand(COMMAND_LAUNCH, async () => {
     try {
- console.debug("current Panel = ", currentPanel, currentPanel?.onDidDispose);
+ console.debug("current Panel = ", currentPanel);
       if (currentPanel) {
         currentPanel.reveal();
         return;
@@ -97,14 +97,6 @@ export async function activate(context: vscode.ExtensionContext) {
       });
       currentPanel = webLoader.getPanel();

- currentPanel?.onDidDispose(
- () => {
- currentPanel = undefined;
- },
- null,
- context.subscriptions
- );
\-
       subscriptions.push(webLoader);
       vscode.window.showInformationMessage("Hello Githru");
     } catch (error) {
```


## 참고자료

- 공식 튜토리얼
  - https://learn.microsoft.com/en-us/azure/ai-services/openai/quickstart
  - https://learn.microsoft.com/ko-kr/azure/ai-services/openai/how-to/chatgpt
  - https://learn.microsoft.com/ko-kr/azure/ai-services/openai/how-to/completions
  - https://learn.microsoft.com/ko-kr/azure/ai-services/openai/how-to/json-mode

- openai v1 마이그레이션 가이드
  - https://github.com/openai/openai-python/discussions/742
  - https://learn.microsoft.com/ko-kr/azure/ai-services/openai/how-to/migration?tabs=python-new%2Cdalle-fix

- 지역별 limit
  - https://learn.microsoft.com/en-us/azure/ai-services/openai/quotas-limits

- 모델별 개요
  - https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models
  - https://platform.openai.com/docs/models

- 모델별 가격정책
  - https://azure.microsoft.com/ko-kr/pricing/details/cognitive-services/openai-service/

- api spec
  - https://github.com/Azure/azure-rest-api-specs
  - https://github.com/Azure/azure-rest-api-specs/tree/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference
  - https://editor.swagger.io/#/?import=https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/preview/2024-05-01-preview/inference.json
  - https://editor.swagger.io/#/?import=https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/2024-02-01/inference.json




In [1]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(), override=True)

openai_api_type = os.environ['OPENAI_API_TYPE']
openai_api_version = os.environ['OPENAI_API_VERSION']
azure_openai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']
azure_openai_api_key = os.environ['AZURE_OPENAI_API_KEY']
azure_openai_deployment_name = os.environ['AZURE_OPENAI_DEPLOYMENT_NAME']

print(f"API VERSION: {openai_api_version}")
print(f"DEPLOYMENT NAME: {azure_openai_deployment_name}")


API VERSION: 2024-02-01
DEPLOYMENT NAME: test-model-20240601


In [2]:
import openai
import langchain

load_dotenv(find_dotenv(), override=True)

print(openai.__version__)
print(langchain.__version__)

# 객체 생성
client = openai.AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
)


# # 질의
# response = client.completions.create(
#   # model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME1"), ## gpt4o (error)
#   model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME2"), ## gpt35-turbo-instruct (ok)
#   # model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME3"), ## gpt35-turbo (ok)
#   prompt="대한민국의 수도는 어디인가요?",
#   max_tokens=100
# )
# print(response)
# print(response.choices[0].text.strip())


# 질의
print(client.chat)
response = client.chat.completions.create(
  model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME1"), ## gpt4o (ok)
  # model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME2"), ## gpt35-turbo-instruct (error, The chatCompletion operation does not work with the specified model)
  # model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME3"), ## gpt35-turbo (ok)
  messages=[{"role": "user", "content": "대한민국의 수도는 어디인가요?"}],
   
  max_tokens=100
)
print(response)
print(response.choices[0].message.content.strip())


1.35.3
0.2.5
ChatCompletion(id='chatcmpl-9fHGojvwoWOkaJxfVsw7M0DUp1Y6J', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='대한민국의 수도는 서울특별시입니다. 서울은 대한민국의 정치, 경제, 문화 중심지로서 중요한 역할을 하고 있습니다.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1719624862, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_abc28019ad', usage=CompletionUsage(completion_tokens=30, prompt_tokens=16, total_tokens=46), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'sever

In [3]:
import openai
import langchain
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

load_dotenv(find_dotenv(), override=True)

print(openai.__version__)
print(langchain.__version__)

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"], ## 2024-02-01
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME1"], ## gpt4o (ok)
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME2"], ## gpt35-turbo-instruct (error, The chatCompletion operation does not work with the specified model, gpt-35-turbo-instruct.)
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME3"], ## gpt35-turbo (ok)
)

message = HumanMessage(
    content="대한민국의 수도는 어디인가요?"
)

response = model.invoke([message])

print(response.content)
print(response.response_metadata)
print(response.response_metadata['model_name'])

1.35.3
0.2.5
대한민국의 수도는 서울입니다. 서울은 대한민국의 정치, 경제, 문화의 중심지로서 많은 주요 정부 기관과 대기업 본사들이 위치해 있습니다. 또한, 역사적 유적지와 현대적 건축물이 공존하는 도시로, 많은 관광객들이 찾는 곳이기도 합니다.
{'token_usage': {'completion_tokens': 66, 'prompt_tokens': 16, 'total_tokens': 82}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}
gpt-4o-2024-05-13


In [4]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

print(prompt)
print(
  prompt.format(country1="대한민국")
)


prompt_partial = prompt.partial(country2="캐나다")
print(prompt_partial)
print(
  prompt_partial.format(country1="대한민국")
)



input_variables=['country1'] partial_variables={'country2': '미국'} template='{country1}과 {country2}의 수도는 각각 어디인가요?'
대한민국과 미국의 수도는 각각 어디인가요?
input_variables=['country1'] partial_variables={'country2': '캐나다'} template='{country1}과 {country2}의 수도는 각각 어디인가요?'
대한민국과 캐나다의 수도는 각각 어디인가요?


In [5]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"], ## 2024-02-01
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME1"], ## gpt4o (ok)
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME2"], ## gpt35-turbo-instruct (error, The chatCompletion operation does not work with the specified model, gpt-35-turbo-instruct.)
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME3"], ## gpt35-turbo (ok)
)

prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": datetime.now().strftime("%B %d")
    },
)

# prompt 생성
prompt.format(n=3)

# chain 을 생성합니다.
chain = prompt | model

# chain 을 실행 후 결과를 확인합니다.
print(
  chain.invoke({"today": "Jan 02", "n": 3}).content
)

1. 가수/배우 이지은 (1989년 1월 2일 출생)
2. 프로게이머 이성진 (별명: Faker, 1996년 5월 7일 출생)
3. 배우 최진혁 (1990년 1월 2일 출생)


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"], ## 2024-02-01
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME1"], ## gpt4o (ok)
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME2"], ## gpt35-turbo-instruct (error, The chatCompletion operation does not work with the specified model, gpt-35-turbo-instruct.)
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME3"], ## gpt35-turbo (ok)
)

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {bot_name} 입니다."),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    bot_name="githru-bot",
    user_input="당신의 이름은 무엇입니까?"
)
print(
  'messages :: ', messages
)

model.invoke(messages).content


messages ::  [SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 githru-bot 입니다.'), HumanMessage(content='당신의 이름은 무엇입니까?')]


'저의 이름은 githru-bot 입니다. 무엇을 도와드릴까요?'

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = AzureChatOpenAI(
    temperature = 0.2,
    openai_api_version=os.environ["OPENAI_API_VERSION"], ## 2024-02-01
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME1"], ## gpt4o (ok)
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME2"], ## gpt35-turbo-instruct (error, The chatCompletion operation does not work with the specified model, gpt-35-turbo-instruct.)
    # azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME3"], ## gpt35-turbo (ok)
)

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 임무는 git commit 내용을 보고 title을 요약하는 것입니다."),
        ("system", """현재 사용하고 있는 커밋 제목 규칙은 아래와 같습니다.

###
git diff 작업내용 1: <브랜치명> 브랜치에서 AAA버그를 수정함
en: fix(<브랜치명>): Bugfix AAA
ko: fix(<브랜치명>): AAA 버그를 수정

git diff 작업내용 2: <브랜치명> 브랜치에서 BBB 기능을 추가함
en: feat(<브랜치명>): Implement BBB
ko: feat(<브랜치명>): BBB 기능 구현

git diff 작업내용 3: <브랜치명> 브랜치에서 CCC 설정를 변경함. 비즈니스 로직 변경 없음.
en: chore(<브랜치명>): Change configuration CCC old -> new.
ko: chore(<브랜치명>): CCC 설정 변경
###"""),
        ("system", "영어로 작성해주세요."),
        ("system", "커밋 제목 아래에 커밋 내용을 요약해서 목록으로 덧붙여주세요."),
        ("human", """아래 git diff 를 보고 위 규칙대로 코드 변경사항이 요약된 커밋 제목을 만들어주세요. 
###
{git_diff}
###"""),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    git_diff="""
diff --git a/packages/view/src/components/Detail/Detail.hook.tsx b/packages/view/src/components/Detail/Detail.hook.tsx
index ac2a121..054a07e 100644
--- a/packages/view/src/components/Detail/Detail.hook.tsx
+++ b/packages/view/src/components/Detail/Detail.hook.tsx
@@ -13,9 +13,9 @@ const useToggleHook = (init = false): UseToggleHook => {

 export const useCommitListHide = (commitNodeListInCluster: CommitNode[]) => {
   const list = getSummaryCommitList(commitNodeListInCluster).reverse();
- const strech = commitNodeListInCluster.reverse();
+ const strech = commitNodeListInCluster.slice(5, commitNodeListInCluster.length).reverse();
   const [toggle, handleToggle] = useToggleHook();
- const commitNodeList = toggle ? strech : list;
+ const commitNodeList = toggle ? [...list, ...strech] : list;

   return {
     toggle,
diff --git a/packages/vscode/src/extension.ts b/packages/vscode/src/extension.ts
index 10e67a4..284870b 100644
--- a/packages/vscode/src/extension.ts
+++ b/packages/vscode/src/extension.ts
@@ -34,7 +34,7 @@ export async function activate(context: vscode.ExtensionContext) {

   const disposable = vscode.commands.registerCommand(COMMAND_LAUNCH, async () => {
     try {
- console.debug("current Panel = ", currentPanel, currentPanel?.onDidDispose);
+ console.debug("current Panel = ", currentPanel);
       if (currentPanel) {
         currentPanel.reveal();
         return;
@@ -97,14 +97,6 @@ export async function activate(context: vscode.ExtensionContext) {
       });
       currentPanel = webLoader.getPanel();

- currentPanel?.onDidDispose(
- () => {
- currentPanel = undefined;
- },
- null,
- context.subscriptions
- );
\-
       subscriptions.push(webLoader);
       vscode.window.showInformationMessage("Hello Githru");
     } catch (error) {

    """,
)
print(
  'messages :: ', messages
)

print(
  'response :: ', model.invoke(messages).content
)


messages ::  [SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 임무는 git commit 내용을 보고 title을 요약하는 것입니다.'), SystemMessage(content='현재 사용하고 있는 커밋 제목 규칙은 아래와 같습니다.\n\n###\ngit diff 작업내용 1: <브랜치명> 브랜치에서 AAA버그를 수정함\nen: fix(<브랜치명>): Bugfix AAA\nko: fix(<브랜치명>): AAA 버그를 수정\n\ngit diff 작업내용 2: <브랜치명> 브랜치에서 BBB 기능을 추가함\nen: feat(<브랜치명>): Implement BBB\nko: feat(<브랜치명>): BBB 기능 구현\n\ngit diff 작업내용 3: <브랜치명> 브랜치에서 CCC 설정를 변경함. 비즈니스 로직 변경 없음.\nen: chore(<브랜치명>): Change configuration CCC old -> new.\nko: chore(<브랜치명>): CCC 설정 변경\n###'), SystemMessage(content='영어로 작성해주세요.'), SystemMessage(content='커밋 제목 아래에 커밋 내용을 요약해서 목록으로 덧붙여주세요.'), HumanMessage(content='아래 git diff 를 보고 위 규칙대로 코드 변경사항이 요약된 커밋 제목을 만들어주세요. \n###\n\ndiff --git a/packages/view/src/components/Detail/Detail.hook.tsx b/packages/view/src/components/Detail/Detail.hook.tsx\nindex ac2a121..054a07e 100644\n--- a/packages/view/src/components/Detail/Detail.hook.tsx\n+++ b/packages/view/src/components/Detail/Detail.hook.tsx\n@@ -13,9 +13,9 @@ co